## Tutorial: Advanced MIP Modelling

Following up on yesterday's sessions on the basics of MIP modelling, in this tutorial we will extend the use of the FICO Xpress and SCIP Python interfaces to more advanced techniques including modelling non-linear problems and the use of controls and callbacks for modifying the behaviour of the solver.

Instructions:
- the tutorial is hands-on, with text descriptions, mathematical formulations, and code snippets shown along the notebook to be analyzed, run and understood. 
- it is divided in 3 parts, which will be solved using the FICO Xpress Python interface. Part 3S, at the end of the notebook, correspond to the first two parts but using the SCIP Python interface.
  - you are expected to solve, in class, the assignments marked as **EXERCISE**.

Software requirements:
- if running in cloud (recommended):
  - https://github.com/features/codespaces (recommended)
  - https://colab.research.google.com/
  - (no Python installation needed, but you need to install "xpress", "PySCIPOpt" packages, and other packages in the remote machine by running "!pip install [module]" - see first code cell)
    - other Python packages needed for this tutorial: matplotlib, networkx, itertools
- if run locally (if you are an expert user):
  - your favorite IDE that supports Interactive Python Notebooks, such as VS Code, PyCharm, or Jupyter notebook
  - Python installation >= 3.9
  - Xpress package >= 9.4
  - PySCIPOpt package >= 5.1
  - other Python packages needed for this tutorial: matplotlib, networkx, itertools

Examples and documentation:
* Check the [Xpress Python API reference manual](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML) and the [Xpress Python examples](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/chExamples.html) online, and find a PDF with training slides named "xpress_python_api.pdf" in the GitHub repo.
* SCIP Python interface: https://github.com/scipopt/PySCIPOpt

Licensing guidelines:
- FICO Xpress Optimizer requires a full license for some of the exercises. You will receive a license file via e-mail, which you should upload into the root directory of your GitHub codespace.
  - an environment variable named XPAUTH_PATH must be set to (the path to) the license file "xpauth.xpr" (automatically done in Codespaces, see code cell below for Colab)
  - <span style="color:red">the provided license must only be used for the purposes of the co@work summer school</span>. Please join the [FICO Academic Partner Program](https://community.fico.com/s/academic-programs) to access full licenses of Xpress free of charge

In [ ]:
# Install Python packages (if running in cloud)
!pip install xpress PySCIPopt matplotlib networkx        # NumPy is installed along with xpress

# If working in Google Colab, uncomment the following lines to set an environment XPAUTH_PATH variable to the license file (assumes the license file is in the same working directory as the notebook)
# import os
# os.environ['XPAUTH_PATH'] = 'xpauth.xpr'

In [ ]:
# To disable warnings regarding the Xpress license, run this code cell
import xpress as xp
import warnings
warnings.simplefilter('ignore', xp.LicenseWarning)

### Part 1 - Modeling non-linear functions as general constraints in MIP problems

The Xpress Optimizer allows for modeling specific non-linear functions so that they are automatically reformulated within a Mixed Integer format. These functions are:

* The absolute value of a function, e.g. $|x|$.
* The maximum and minimum of a list of functions, e.g. $\max(x+1, 2y-2,...)$.
* The logical And and Or operators (only work on binary variables) on a set of functions, for example $(x \wedge y) \vee z$.

The Xpress Python interface allows for natural modeling of these functions by using the operators `xpress.abs`, `xpress.max`, `xpress.min`, `xpress.And`, `xpress.Or`, and the binary Python operators `&` and `|`:
* `xp.abs(2*x-4)` for the expression $|2x-4|$.
* `xp.min(x1, x2, 2*x3-1)` for the expression $\min(x_1, x_2, 2x_3-1)$, and similar for the maximum.
* `xp.And(y1, y2, y3, 1-y4)` for the logical AND operators between the binary variables $y_1$, $y_2$, $y_3$, and the negation of $y_4$, which can be modeled as $1-y_4$ as it is 0 when $y_4=1$ and viceversa, functioning therefore as a negation operator. Another possibility would be to model it in Python as `y1 & y2 & y3 & (1-y4)`. The same logic applies for `xp.Or` and the Python `|` operator.

Let's see an example of this for the following non-linear problem:

$$
\begin{array}{ll}
\min & |x-y| + 2.\hbox{max}(2x + 4, 3y + 2x)\\
\textrm{s.t.} & x+2y \geq 20 \\
& x+3y \leq 25
\end{array}
$$

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

p = xp.problem()

x = p.addVariable()
y = p.addVariable()

p.setObjective(xp.abs(x-y) + 2 * xp.max(2*x + 4, 3*y + 2*x))

p.addConstraint(x+2*y >= 20, x+3*y <= 25)

p.optimize()

print(p.getSolution(x), p.getSolution(y))

As you can observe in the solver logs, the problem has been reformulated as a MIP, and solved by the Xpress MIP Optimizer.

##### MIP reformulation of $|x|$  and AND/OR



Below is how we would have to model the constraint $z = |x-y|$ if we did not have the `xp.abs` function within Xpress. We would have to introduce a binary variable $w$ and writing the following constraints, two of which are indicator constraints:

$$
\begin{array}{lll}
z \ge x-y\\
z \ge y-x\\
w = 1 \Rightarrow z \le x-y\\
w = 0 \Rightarrow z \le y-x\\
w \in \{0,1\}
\end{array}
$$

AND/OR operators do not require extra variables: for an operation $z = x \wedge y$, with $x$ and $y$ binary variables, one can write:

$$
\begin{array}{l}
z \le x\\
z \le y\\
z \ge x + y - 1
\end{array}
$$

and similarly for an operation $z = x \vee y$:

$$
\begin{array}{l}
z \ge x\\
z \ge y\\
z \le x + y
\end{array}
$$

**EXERCISE 1**: Implement the model with a logical constraint below (using either Xpress or Python native operators), solve it and print the optimal solution vector:
$$
\min \sum_{i \in \mathcal{Z}} z_i
$$

subject to:

$$
(x \wedge z_0) + x \wedge y \wedge (z_1 \vee z_2 \vee z_3) = 1 \\
x, y, z_i \in \{0,1\}, \quad \forall i \in \mathcal{Z}
$$

with $\mathcal{Z} = \{0,...,3\}$.

In [ ]:
# FICO Xpress Optimizer - Python Interface (solution)


### Part 2 - Tuning and solving a MIP from MIPLIB using solver controls and attributes

Let's move on to solving larger instances, like a problem instance from the [MIPLIB 2017 collection](https://miplib.zib.de/tag_collection.html). Specifically, [roi2alpha3n4](https://miplib.zib.de/instance_details_roi2alpha3n4.html). 

*Please download the instance file by clicking on [roi2alpha3n4.mps.gz](https://miplib.zib.de/WebData/instances/roi2alpha3n4.mps.gz) and move the file into the working directory of this notebook*.

Run the code cell below to have the Xpress Optimizer solve the problem.

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

filename = 'roi2alpha3n4.mps.gz'

p = xp.problem()

p.read(filename)

p.controls.timelimit = 30       # set time limit, in seconds

p.optimize()

More than 320 [control parameters](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/chapter7.html) and 140 [problem attributes](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/chapter8.html) exist within the Xpress Optimizer to govern the solution procedure and query problem features, respectively. 

A **control** is a parameter that can influence the behavior (and therefore the performance) of the Xpress Optimizer:
  - for example: the MIP gap target, the feasibility tolerance, or the type of root LP algorithms are controls that can be defined by the user
  - although the default control values have been found to work well in practice over a range of problems, they may be tuned to enhance performance of the Optimizer for a specific problem 
  - problem controls can both be read from and written to an optimization problem. The majority of these take integer values and act as switches between various types of behavior
  
An **attribute** is a feature of an optimization problem, such as the number of rows and columns or the number of quadratic elements in the objective function:
  - they are read-only parameters, i.e. their value cannot be directly modified by the user
  - helpful to query the solve and solution statuses of a run, print the objective value or the time spent solving instead of analyzing solver logs

Examples of problem controls that **impact the stopping criteria** includes:
- [Time limit](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/TIMELIMIT.html): The maximum time in seconds that the Optimizer will run before it terminates, including the problem setup time and solution time.
- [Solution time limit](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/SOLTIMELIMIT.html): The maximum time in seconds that the Optimizer will run a MIP solve before it terminates, given that a solution has been found. As long as no solution has been found, this control will have no effect.
- [Relative optimality gap tolerance](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/MIPRELSTOP.html): This determines when the branch and bound tree search will terminate. For example, to stop the tree search when a MIP solution has been found and the Optimizer can guarantee it is within 5% of the optimal solution, set MIPRELSTOP to 0.05.
- [Absolute optimality gap tolerance](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/MIPABSSTOP.html): The absolute tolerance determining whether the tree search will continue or not. For example, to stop the tree search when a MIP solution has been found and the Optimizer can guarantee it is within 100 of the optimal solution, set MIPABSSTOP to 100.

**EXERCISE 2.1**: Run the code cell below and identify which stopping criteria triggered the end of the optimization procedure by analyzing the output generated by the querying of problem attributes. Then set different values for the other controls to have them stop the optimization as well.

<i>Note that there is solver control that allows you to turn off output logs.</i>

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

p = xp.problem()

p.read(filename)

# Controls that impact the stopping criteria
p.controls.timelimit = 40       # set time limit, in seconds, regardless of whether a solution has been found
p.controls.soltimelimit = 30    # set time limit, in seconds, given that a solution has been found
p.controls.miprelstop = 0.1    # relative optimality gap tolerance
p.controls.mipabsstop = 0.25     # absoute optimality gap tolerance

# p.controls.outputlog = 0        # turn off logging from Xpress

p.optimize()

# Query and print problem attributes
print("\nSolve status: ", p.attributes.solvestatus.name)            # UNSTARTED, STOPPED, FAILED, or COMPLETED
print("Solution status: ", p.attributes.solstatus.name)             # NOTFOUND, OPTIMAL, FEASIBLE, INFEASIBLE, or UNBOUNDED
print("The problem has", p.attributes.rows, "rows and", p.attributes.cols, "columns")

print("Objective value:", round(p.attributes.objval,2))
print("Time spent:", p.attributes.time, " sec.")
print("Primal-dual integral:", round(p.attributes.primaldualintegral,2))
print("Number of nodes solved:", p.attributes.nodes)


A few of the wide range of problem controls that **can have an impact on the solver performance** are:

- [Cut strategy](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/CUTSTRATEGY.html): This specifies the cut strategy. A more aggressive cut strategy, generating a greater number of cuts, will result in fewer nodes to be explored, but with an associated time cost in generating the cuts.
- [Heuristic emphasis](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/HEUREMPHASIS.html): This control specifies an emphasis for the search w.r.t. primal heuristics and other procedures that affect the speed of convergence of the primal-dual gap. This setting triggers many additional heuristic calls, aiming for reducing the gap at the beginning of the search, typically at the expense of an increased time for proving optimality.
- [Presolve](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/PRESOLVE.html): This control determines whether presolving should be performed prior to starting the main algorithm. Presolve attempts to simplify the problem by detecting and removing redundant constraints, tightening variable bounds, etc. In some cases, infeasibility may even be determined at this stage, or the optimal solution found.
- [Symmetry](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/SYMMETRY.html): Adjusts the overall amount of effort for symmetry detection.
- [Heuristic search effort](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/HEURSEARCHEFFORT.html): Adjusts the overall effort level spent by local search heuristics.

**EXERCISE 2.2**: Experiment with the values of the solver controls in the TO DO section below and try to find a configuration (combination of control values) that improves the performance of the solver for a time limit of 30 seconds, comparing to the previous run (default values), **in at least one of the following metrics to be minimized**:
1) the relative optimality gap at the end of the run
2) [primal-dual integral](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/PRIMALDUALINTEGRAL.html)
3) [number of nodes](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/NODES.html) explored by the B&B process

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

p = xp.problem()

p.read(filename)

# Controls that impact the solver performance
# TO DO: experiment with the controls below and find a configuration that beats the default values (previous run)
p.controls.cutstrategy = -1     # integer, default -1
p.controls.heuremphasis = -1    # integer, default -1
p.controls.presolve = 1         # integer, default 1
p.controls.symmetry = 1         # integer, default 1
p.controls.heursearcheffort = 1 # double, default 1

p.controls.soltimelimit = 30

p.optimize()

# Query and print problem attributes
print("\nSolve status: ", p.attributes.solvestatus.name)            # UNSTARTED, STOPPED, FAILED, or COMPLETED
print("Solution status: ", p.attributes.solstatus.name)             # NOTFOUND, OPTIMAL, FEASIBLE, INFEASIBLE, or UNBOUNDED
print("The problem has", p.attributes.rows, "rows and", p.attributes.cols, "columns")

print("Objective value:", round(p.attributes.objval,2))
print("Time spent:", p.attributes.time, " sec.")
print("Primal-dual integral:", round(p.attributes.primaldualintegral,2))
print("Number of nodes solved:", p.attributes.nodes)


### Coffee break

### Part 3 - Advanced techniques for solving the TSP (Xpress)

In this last part, we will solve the [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) using different modeling techniques.

<i>Note that this is just an academic exercise done for demonstration purposes, with no intention to propose or claim new solution methods for solving the TSP. Moreover, we refer to the state-of-the-art solver for the TSP and other related problems [Concorde TSP solver](https://www.math.uwaterloo.ca/tsp/concorde.html).</i>

##### Standard formulation


Let's start with the standard formulation, where $x_{ij} \in \{0,1\}, \forall i,j \in \mathcal{N}$ represents if the tour uses the edge $(i,j)$ (i.e. if we go from city $i$ to $j$) in the final solution. An optimal tour can be found by solving:
$$
\min \sum_{i,j \in \mathcal{N}} c_{ij} x_{ij}
$$

subject to:

* We have to enter and leave every city, and source node cannot be the same as the destination node: 
$$
\sum_{j \in \mathcal{N}} x_{ij} = 1, \quad \forall i \in \mathcal{N} \\
\sum_{j \in \mathcal{N}} x_{ji} = 1, \quad \forall i \in \mathcal{N} \\
x_{ii} = 0, \quad \forall i \in \mathcal{N}
$$

We overlook subtour elimination constraints for now.

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp
import numpy as np

'''Create random TSP problem data'''
n = 17
CITIES = range(n)  # set of cities: 0..n-1

np.random.seed(0)

X = 100 * np.random.rand(n)
Y = 100 * np.random.rand(n)

XY = (X, Y)

# Compute distance matrix
dist = np.ceil(np.sqrt ((X.reshape(n,1) - X.reshape(1,n))**2 +
                        (Y.reshape(n,1) - Y.reshape(1,n))**2))

# Create problem
p = xp.problem()

# Create variables as a square matrix of binary variables. Note
# the use of p.addVariables to ensure NumPy uses the Xpress operations for handling these vectors.
x = p.addVariables(n,n, vartype=xp.binary, name='x')

# Degree constraints
p.addConstraint(xp.Sum(x[i,:]) == 1  for i in CITIES)
p.addConstraint(xp.Sum(x[:,i]) == 1  for i in CITIES)

# Fix diagonals (i.e. city X -> city X) to zero
p.addConstraint(x[i,i] == 0 for i in CITIES)

# Objective function
p.setObjective (xp.Sum((dist * x).flatten()))

p.optimize()

Now let's use matplotlib and networkx to visualize the solution for the Xpress interface and check if there are any subtours:

In [ ]:
# Visualize solution using networkx and matplotlib
import networkx as nx 
import matplotlib.pyplot as plt
import itertools

def plot_sol(p):

    # collect the edges: if the value of x[i,j] is 1, then the edge (i,j) is in the solution
    edges = []
    for (i,j) in itertools.permutations(CITIES, 2):
        if p.getSolution(x[i,j]) > 0.5: # variable is binary so > 0.5 --> is 1
            edges.append( (i,j) )

    # create dictionary with coordinates for each node to plot the graph 
    xy = {}
    for i in range(n):
        xy[i] = (X[i], Y[i])

    # make figure look nicer
    plt.figure(figsize=(10,6), dpi=100)

    # create empty graph
    optgraph = nx.Graph()

    # add edges
    optgraph.add_edges_from(edges)

    # draw the nodes, with labels in the position xy (see when we read the instance)
    nx.draw(optgraph, node_size=300, pos=xy, with_labels=True, node_color='lightblue')

    # show drawing
    plt.show()

# Plot solution
if p.attributes.solstatus not in [xp.SolStatus.OPTIMAL, xp.SolStatus.FEASIBLE]:
    print("Solve status:", p.attributes.solvestatus.name)
    print("Solution status:", p.attributes.solstatus.name)
else:
    plot_sol(p)

As you can see, the solution contains subtours, and therefore it is not a valid solution for problem yet. We need to add subtour elimination constraints, that is, prevent each and every possible subtour from ocurring.

**EXERCISE 3.1**: Add the following subtour elimination constraints to the problem, solve it and plot the solution:

$$
\sum_{i,j \in S} x_{ij} \leq |S| - 1 \quad \forall S \subsetneq N,\ |S| \geq 2
$$

In [ ]:
# FICO Xpress Optimizer - Python Interface (solution)



The solution is now valid and complete, since an Hamiltonian tour (no subtours) has been formed. However, you could experience that the problem building and solving times are rather high for a TSP with only 17 cities.

This is due to the complete enumeration of all possible subtours, whose number grows exponentially with the instance size making the problem creation and solving times lengthy when problems become realistically large.

##### Formulation using the Miller, Tucker, Zemlin subtour elimination constraints

Now we will model the same problem but using the Miller, Tucker, Zemlin subtour elimination constraints, which requires a new set of continuous variables:

$t_i$ = step at which node $i$ is visited, $\forall i = {1,...,|\mathcal{N}|}$

which are used as auxiliary variables for the formulation of subtour elimination constraints. Although a new set of real variables of size $|\mathcal{N}|$ is introduced, we are able to prevent all subtours by introducing a set of constraints of size $(|\mathcal{N}|-1)^2$, considerably reducing the problem size and complexity when comparing with the previous formulation:

$$
\min \sum_{i,j \in \mathcal{N}} c_{ij} x_{ij}
$$

subject to:

* We have to enter and leave every city, and source node cannot be the destination node for any move: 
$$
\sum_{j \in \mathcal{N}} x_{ij} = 1, \quad \forall i \in \mathcal{N} \\
\sum_{j \in \mathcal{N}} x_{ji} = 1, \quad \forall i \in \mathcal{N} \\
x_{ii} = 0, \quad \forall i \in \mathcal{N}
$$
* Miller, Tucker, Zemlin subtour elimination constraints:
$$
t_j \geq t_i + 1 - (n-1)*(1 - x_{ij}), \forall i,j = {1,...,|\mathcal{N}|}
$$

with $n = |\mathcal{N}|$.

**EXERCISE 3.2**: Adjust the previous model to represent the Miller, Tucker, Zemlin formulation of the TSP

In [ ]:
# FICO Xpress Optimizer - Python Interface (solution)


As you can confirm by the graph and solver logs, this formulation yields the same solution as the previous one, but solves the problem much faster. Let's try to beat that in the next section!

##### Using callbacks

Now let's try to speed up the problem creation and solving times even more by using **solver callbacks** to add only those subtour elimination constraints that are really needed during the solving process using the standard formulation.

Callback functions are user–defined routines to be specified to the Optimizer which should be called at various stages during the optimization process, prompting the Optimizer to return to the user's program before continuing with the solution algorithm. They are a useful tool particularly for Mixed Integer Programming (MIP) problems by allowing the user to "interact" with the solver during the solution search process (Branch-and-Bound). 

[Using callback functions](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/chCallbacks.html) is simple: 
  * the user first defines a function (say *myfunction*) that is to be run every time the branch-and-bound reaches a well-specified point
  * secondly, the user calls a function (such as [problem.addcbpreintsol](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addcbpreintsol.html)) with *myfunction* as its argument
  * finally, the user runs the optimize command that launches the branch-and-bound, the simplex solver, or the barrier solver; it is while these are run that myfunction is called at the intended points (depending on the selected callback type, check the [list of callbacks accepted by the Xpress Optimizer](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/HTML/chapter5.html?scroll=section5002))

In our TSP exercise, we will relax (leave out) the subtour elimination constraints, and instead add a callback funtion that is triggered every time an integer (feasible) solution to the relaxed problem is found during the B&B process by using the [problem.addcbpreintsol](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addcbpreintsol.html) solver callback, instructing the solver whether to accept (if no subtours) or discard (subtours exist) the solution found, and add the corresponding subtour elimination constraints to the problem in the latter case. Therefore, the user-defined callback function needs to perform two main tasks:
  * checking whether a given solution forms a Hamiltonian tour
  * separating subtour elimination constraints from the current node solution

**EXERCISE 3.3**: analyze and understand the callback function below, and complete the "TO DO" section on the bottom by calling the function [problem.addcuts](https://www.fico.com/fico-xpress-optimization/docs/dms2024-03/solver/optimizer/python/HTML/problem.addcuts.html) with the appropriate arguments. You will need to run the subsequent cell to check whether the callback function works properly or not.


In [55]:
# FICO Xpress Optimizer - Python Interface

# Define callback function
def cb_preintsol(prob, data, soltype, cutoff):
    '''Callback for checking if solution is acceptable
    prob: Xpress problem object = TSP model without subtour elimination
    data: data object = number of cities
    soltype: type of MIP solution that has been found. 0 - LP relaxation is integer feasible, 1 - MIP solution found by a heuristic, 2 - MIP solution provided by the user
    '''

    n = data                             # number of cities
    xsol=[]                              # array to store integer solution found by heuristics or during the branch and bound search
    prob.getlpsol(x=xsol)                # get array with values of x variables in the current solution
    xsol = np.array(xsol).reshape(n,n)   # convert into matrix-shaped np array
    tour = np.argmax(xsol, axis=1)       # get indices of the max (non-zero) element in each column = resulting tour

    i = 0       # to store next city
    ncities = 1 # to count number of cities visited

    # Scan cities in order until we get back to 0 or the solution is wrong. 
    while tour[i] != 0 and ncities < n:
        ncities += 1
        i = tour[i]

    reject = False # at this point do not reject the solution
    if ncities < n:
        # The tour given by the current solution does not pass through
        # all the nodes and thus contains subtours.
        # If soltype is non-zero, the solution was found by a heuristic
        # or provided by the user, se we reject by setting reject=True without further branching.
        # If instead soltype is zero, the solution came from the LP relaxation  
        # of the current B&B node which was found to be integer feasible. In this case 
        # we will reject but by adding a cut that cuts off that solution.
        # Note that we must NOT set reject=True in that case because that would result in just
        # dropping the node, no matter whether we add cuts or not.
        if soltype != 0:
            reject = True
        else:
            # Obtain an order by checking the maximum of the variable matrix
            # for each row
            unchecked = np.zeros(n) # np array of zeros of size n to assign each city to a subtour
            nsubtour = 0            # to number and identify subtours

            # Initialize the vectors to be passed to problem.addcuts
            cut_mstart = [0]
            cut_ind = []
            cut_coe = []
            cut_rhs = []

            nnz = 0   # to mark the start of the variables indices for each cut
            ncuts = 0 # to count number of cuts (subtours)

            # while there are cities that are not assigned to a subtour
            while np.min(unchecked) == 0:
                nsubtour += 1                    # current subtour id
                firstcity = np.argmin(unchecked) # first city (index) in unchecked that is not assigned a subtour yet (i.e., =0)
                i = firstcity 

                # Scan cities in order for each subtour
                while True:
                    unchecked[i] = nsubtour  # mark city i as belonging to current subtour
                    i = tour[i]              # proceed to next city in the current subtour

                    if i == firstcity:       # if next city is the first city, subtour is finished, stop
                        break

                # Find indices of all variables with origin in this subtour and destination
                # outside this subtour. We will force the sum of those variables to be >= 1 
                # (i.e. subtour will be constrained after the cut is added). S is the set of nodes
                # traversed by the subtour, compS is its complement (list of nodes without S)
                S     = np.where(unchecked == nsubtour)[0].tolist()
                compS = np.where(unchecked != nsubtour)[0].tolist()
                indices = [i*n+j for i in S for j in compS] # convert into decision variables array indices

                # We need to presolve new row to get data for addcuts later (presolved problem)
                mcolsp, dvalp = [], [] # lists to be populated by the 'presolverow' method

                # Presolve new row (cut) in order to add it to the presolved problem via addcuts
                drhsp, status = prob.presolverow(rowtype='G',
                                                    origcolind=indices,
                                                    origrowcoef=np.ones(len(indices)),
                                                    origrhs=1,
                                                    maxcoefs=prob.attributes.cols,
                                                    colind=mcolsp,
                                                    rowcoef=dvalp)

                nnz += len(mcolsp)
                ncuts += 1

                cut_ind.extend(mcolsp) # array which will be filled with the columns of the presolved row
                cut_coe.extend(dvalp)  # array which will be filled with the coefficients of the presolved row
                cut_rhs.append(drhsp)  # array containing the right hand side elements for the cuts
                cut_mstart.append(nnz) # array containing offset into the colind and cutcoef arrays indicating the start of each cut

            # add cuts
            if ncuts > 0:
                # **TO DO** - call prob.addcuts and pass the necessary arguments to add the subtour elimination constraints (list of cuts)
                

    return (reject, None)

Now we will solve the problem with the standard formulation (without subtour elimination constraints), but **adding the callback function** and plotting the solution to validate the approach. 

Run the code cell, and verify that the optimal solution values remain the same amongst the three different methods of solving the problem, and compare the running times needed to achieve the optimal solution. You may need to debug/correct the callback function definition in the previous code cell.

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

# Create problem
p = xp.problem()

# Create variables as a square matrix of binary variables. Note
# the use of p.addVariables to ensure NumPy uses the Xpress operations for handling these vectors.
x = p.addVariables(n,n, vartype=xp.binary, name='x')

# Degree constraints
p.addConstraint(xp.Sum(x[i,:]) == 1  for i in CITIES)
p.addConstraint(xp.Sum(x[:,i]) == 1  for i in CITIES)

# Fix diagonals (i.e. city X -> city X) to zero
p.addConstraint(x[i,i] == 0 for i in CITIES)

# Objective function
p.setObjective (xp.Sum((dist * x).flatten()))

# Add callback function
p.addcbpreintsol(cb_preintsol, n)

# Solve the problem and print solution
p.optimize()

print(p.attributes.solstatus.name)

if p.attributes.solstatus not in [xp.SolStatus.OPTIMAL, xp.SolStatus.FEASIBLE]:
    print("Solve status:", p.attributes.solvestatus.name)
    print("Solution status:", p.attributes.solstatus.name)
else:
    plot_sol(p)  # plot solution

As we can observe, the problem has been solved even faster than using the MTZ formulation, yielding the same solution and objective value in considerably less time, making the problem even more scalable. 

**BONUS EXERCISE 3.4**: experiment (increase) the number of cities (n) in the cell below and run the previous code cell again to see how high 'n' can go before the problem becomes computationally too expensive (let's say, takes more than 1 minute to solve to proven optimality).

Compare with results from MTZ formulation for the same instance sizes.

In [ ]:
'''Create random TSP problem data'''
n = 17
CITIES = range(n)  # set of cities: 0..n-1

np.random.seed(0)

X = 100 * np.random.rand(n)
Y = 100 * np.random.rand(n)

XY = (X, Y)

# Compute distance matrix
dist = np.ceil(np.sqrt ((X.reshape(n,1) - X.reshape(1,n))**2 +
                        (Y.reshape(n,1) - Y.reshape(1,n))**2))

### Part 3S - Advanced techniques for solving the TSP (SCIP)

In this last part, we will solve the [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) using different modeling techniques.

<i>Note that this is just an academic exercise done for demonstration purposes, with no intention to propose or claim new solution methods for solving the TSP. Moreover, we refer to the state-of-the-art solver for the TSP and other related problems [Concorde TSP solver](https://www.math.uwaterloo.ca/tsp/concorde.html).</i>

##### Standard formulation


Let's start with the standard formulation, where $x_{ij} \in \{0,1\}, \forall i,j \in \mathcal{N}$ represents if the tour uses the edge $(i,j)$ (i.e. if we go from city $i$ to $j$) in the final solution. An optimal tour can be found by solving:
$$
\min \sum_{i,j \in \mathcal{N}} c_{ij} x_{ij}
$$

subject to:

* We have to enter and leave every city, and source node cannot be the destination node for each move: 
$$
\sum_{j \in \mathcal{N}} x_{ij} = 1, \quad \forall i \in \mathcal{N} \\
\sum_{j \in \mathcal{N}} x_{ji} = 1, \quad \forall i \in \mathcal{N} \\
x_{ii} = 0, \quad \forall i \in \mathcal{N}
$$

We overlook subtour elimination constraints for now.

In [41]:
# SCIP - Python Interface

from pyscipopt import Model, quicksum
import numpy as np
import itertools

'''Create random TSP problem data'''
n = 15
CITIES = range(n)  # set of cities: 0..n-1

np.random.seed(0)

X = 100 * np.random.rand(n)
Y = 100 * np.random.rand(n)

XY = (X, Y)

# Compute distance matrix
dist = np.ceil(np.sqrt ((X.reshape(n,1) - X.reshape(1,n))**2 +
                        (Y.reshape(n,1) - Y.reshape(1,n))**2))

# Create problem
model = Model()

# Create variables as a square matrix of binary variables. Note
# the use of p.addVariables to ensure NumPy uses the Xpress operations for handling these vectors.
x = {}
for i in range(n):
    for j in range(n):
        x[i,j] = model.addVar(vtype="B", name='x[%i,%i]'%(i,j))

# Degree constraints
for i in CITIES:
    model.addCons(quicksum(x[i,j] for j in CITIES) == 1)
    model.addCons(quicksum(x[j,i] for j in CITIES) == 1)

# Fix diagonals (i.e. city X -> city X) to zero
for i in CITIES:
    model.addCons(x[i,i] == 0)

# Objective function
model.setObjective(quicksum(dist[i,j]*x[i,j] for i in range(n) for j in range(n)))

model.optimize()

Now let's use matplotlib and networkx to visualize the solution for the Xpress interface and check if there are any subtours:

In [ ]:
# Visualize solution using networkx and matplotlib
import networkx as nx 
import matplotlib.pyplot as plt

def plot_sol(model):

    # collect the edges: if the value of x[i,j] is 1, then the edge (i,j) is in the solution
    edges = []
    for (i,j) in itertools.permutations(CITIES, 2):
        if model.getVal(x[i,j]) > 0.5: # variable is binary so > 0.5 --> is 1
            edges.append( (i,j) )

    # create dictionary with coordinates for each node to plot the graph 
    xy = {}
    for i in range(n):
        xy[i] = (X[i], Y[i])

    # make figure look nicer
    plt.figure(figsize=(10,6), dpi=100)

    # create empty graph
    optgraph = nx.Graph()

    # add edges
    optgraph.add_edges_from(edges)

    # draw the nodes, with labels in the position xy (see when we read the instance)
    nx.draw(optgraph, node_size=300, pos=xy, with_labels=True, node_color='lightblue')

    # show drawing
    plt.show()

# Plot solution
if model.getNSols() == 0:
    print("No feasible solution found.")
else:
    plot_sol(model)  # print solution and cost

As you can see, the solution contains subtours, and therefore it is not a valid solution for problem yet. We need to add subtour elimination constraints, that is, prevent each and every possible subtour from ocurring.

Let's add the subtour elimination constraints to the problem:

$$
\sum_{i,j \in S} x_{ij} \leq |S| - 1 \quad \forall S \subsetneq N,\ |S| \geq 2
$$

In [ ]:
# SCIP - Python Interface

model.freeTransform()

# Add subtour elimination constraints, solve and print
for L in range(2,len(CITIES)):
    for subset in itertools.combinations(CITIES, L):
        model.addCons(quicksum(x[i,j] for i in subset for j in subset) <= len(subset) - 1)        


model.setParam("limits/time", 20)
model.optimize()

# Plot solution
if model.getNSols() == 0:
    print("No feasible solution found.")
else:
    plot_sol(model) 

The solution is now valid and complete, since an Hamiltonian tour (no subtours) has been formed. However, you could experience that the problem building and solving times are rather high for a TSP with only 15 cities.

This is due to the complete enumeration of all possible subtours, whose number grows exponentially with the instance size making the problem creation and solving times lengthy when problems become realistically large.

##### Formulation using the Miller, Tucker, Zemlin subtour elimination constraints


Now we will model the same problem but using the Miller, Tucker, Zemlin subtour elimination constraints, which requires a new set of continuous variables:

$t_i$ = step at which node $i$ is visited, $\forall i = {1,...,|\mathcal{N}|}$

which are used as auxiliary variables for the formulation of subtour elimination constraints. Although a new set of real variables of size $|\mathcal{N}|$ is introduced, we are able to prevent all subtours by introducing a set of constraints of size $(|\mathcal{N}|-1)^2$, considerably reducing the problem size and complexity when comparing with the previous formulation:

$$
\min \sum_{i,j \in \mathcal{N}} c_{ij} x_{ij}
$$

subject to:

* We have to enter and leave every city, and source node cannot be the destination node for any move: 
$$
\sum_{j \in \mathcal{N}} x_{ij} = 1, \quad \forall i \in \mathcal{N} \\
\sum_{j \in \mathcal{N}} x_{ji} = 1, \quad \forall i \in \mathcal{N} \\
x_{ii} = 0, \quad \forall i \in \mathcal{N}
$$
* Miller, Tucker, Zemlin subtour elimination constraints:
$$
t_j \geq t_i + 1 - (n-1)*(1 - x_{ij}), \forall i,j = {1,...,|\mathcal{N}|}
$$

with $n = |\mathcal{N}|$.

In [ ]:
# SCIP - Python Interface

from pyscipopt import Model, quicksum
import numpy as np

from pyscipopt import Model, quicksum
import numpy as np

# Create problem
model = Model()

# Create variables as a square matrix of binary variables. Note
# the use of p.addVariables to ensure NumPy uses the Xpress operations for handling these vectors.
x = {}
t = {}
for i in range(n):
    for j in range(n):
        x[i,j] = model.addVar(vtype="B", name='x[%i,%i]'%(i,j))
    t[i] = model.addVar(name='t[%i]'%i)

# Degree constraints
for i in CITIES:
    model.addCons(quicksum(x[i,j] for j in CITIES) == 1)
    model.addCons(quicksum(x[j,i] for j in CITIES) == 1)

# Fix diagonals (i.e. city X -> city X) to zero
for i in CITIES:
    model.addCons(x[i,i] == 0)

# Miller, Tucker, Zemlin subtour elimination constraints
for i in range(1,n):
    for j in range(1,n):
        model.addCons(t[j] >= t[i] + 1 - n*(1 - x[i,j]))

# Objective function
model.setObjective(quicksum(dist[i,j]*x[i,j] for i in range(n) for j in range(n)))

model.optimize()

# Plot solution
if model.getNSols() == 0:
    print("No feasible solution found.")
else:
    plot_sol(model)  # print solution and cost



As you can confirm by the graph and solver logs, this formulation yields the same solution as the previous one, but solves the problem much faster.